# 2차시 모델을 이용한 파라미터 변경 cpu


```py
# 1차시

img_size = 128
batch_size = 64

calsses_num = 4

```

```py
# 2차시
    model.add(Conv2D(filters = 256, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu', input_shape = (img_size,img_size,3), kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 128, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters = 64(변경), kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters = 32(변경), kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters = 32(변경), kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
```

In [6]:
# from tensorflow.python.client import device_lib

# print(device_lib.list_local_devices())

In [7]:
import pandas as pd
import numpy as np

import glob,scipy,os
import matplotlib.pyplot as plt

import seaborn as sns
#from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix



import keras
from keras import backend as K
import tensorflow as tf

from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


In [9]:
# 변수 
model_name = "param_2"
model_save = f'{model_name}.hdf5'

PATH="C:\FoodClassification\data"
img_size = 128
batch_size = 64
calsses_num = 4

In [10]:
# train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.33)

train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 shear_range=0.2,
                 zoom_range=0.2,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,
                validation_split=0.33
                                  )

In [11]:
train_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        class_mode = 'categorical',
        subset="training")


val_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(img_size,img_size), 
        batch_size=batch_size,
        class_mode = 'categorical',
        subset="validation")


Found 13350 images belonging to 4 classes.
Found 6574 images belonging to 4 classes.


In [5]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [12]:
model = Sequential()

model.add(Conv2D(filters = 256, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu', input_shape = (img_size,img_size,3), kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 32, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 32, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
# Dense
model.add(Dense(128, activation = "relu",kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(128, activation = "relu",kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(calsses_num, activation = "softmax",kernel_initializer='he_normal',kernel_regularizer=l2()))
model.summary()

#callbacks
checkpointer = ModelCheckpoint(filepath=model_save, verbose=1, save_best_only=True, save_weights_only=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, mode='auto')
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, mode='auto')
# stpes_per_epoch = 2250/64


model.compile(optimizer = 'Adamax' , loss = "categorical_crossentropy", metrics=["accuracy"])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 256)       19456     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       819328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)       0         
______________________________________

In [0]:
# import os
# tpu_model = tf.contrib.tpu.keras_to_tpu_model(
#     model,
#     strategy=tf.contrib.tpu.TPUDistributionStrategy(
#         tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#     )
# )

In [0]:
    # opt = Adam(lr=0.0002, beta_1=0.5)
    # model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
    # model.compile(optimizer = 'rmsprop' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [13]:
history = model.fit_generator(train_generator,steps_per_epoch=100,
                        validation_data=val_generator,validation_steps=100, 
                        epochs=100)


Epoch 1/100
 36/100 [=========>....................] - ETA: 4:26 - loss: 1.5844 - accuracy: 0.3650

C:\Users\admin\Anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 99/100 [============================>.] - ETA: 4s - loss: 1.4272 - accuracy: 0.4116

C:\Users\admin\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


100/100 [==============================] - 608s 6s/step - loss: 1.4279 - accuracy: 0.4114 - val_loss: 1.5562 - val_accuracy: 0.2297
Epoch 2/100
100/100 [==============================] - 594s 6s/step - loss: 1.2024 - accuracy: 0.5131 - val_loss: 1.3505 - val_accuracy: 0.4669
Epoch 3/100
100/100 [==============================] - 590s 6s/step - loss: 1.0770 - accuracy: 0.5611 - val_loss: 1.5441 - val_accuracy: 0.4528
Epoch 4/100
100/100 [==============================] - 589s 6s/step - loss: 0.9989 - accuracy: 0.6128 - val_loss: 1.0728 - val_accuracy: 0.5573
Epoch 5/100
100/100 [==============================] - 588s 6s/step - loss: 0.9663 - accuracy: 0.6265 - val_loss: 1.3466 - val_accuracy: 0.5447
Epoch 6/100
100/100 [==============================] - 588s 6s/step - loss: 0.8948 - accuracy: 0.6533 - val_loss: 1.2875 - val_accuracy: 0.5456
Epoch 7/100
100/100 [==============================] - 590s 6s/step - loss: 0.8663 - accuracy: 0.6716 - val_loss: 0.8108 - val_accuracy: 0.5777
Epoc

100/100 [==============================] - 602s 6s/step - loss: 0.3356 - accuracy: 0.8812 - val_loss: 0.8030 - val_accuracy: 0.6412
Epoch 58/100
100/100 [==============================] - 603s 6s/step - loss: 0.3059 - accuracy: 0.8905 - val_loss: 0.9697 - val_accuracy: 0.7321
Epoch 59/100
100/100 [==============================] - 605s 6s/step - loss: 0.2957 - accuracy: 0.8954 - val_loss: 1.0537 - val_accuracy: 0.6963
Epoch 60/100
100/100 [==============================] - 605s 6s/step - loss: 0.3014 - accuracy: 0.8930 - val_loss: 0.6763 - val_accuracy: 0.7128
Epoch 61/100
100/100 [==============================] - 606s 6s/step - loss: 0.2963 - accuracy: 0.8908 - val_loss: 1.0806 - val_accuracy: 0.7043
Epoch 62/100
100/100 [==============================] - 616s 6s/step - loss: 0.2937 - accuracy: 0.8948 - val_loss: 1.5715 - val_accuracy: 0.5133
Epoch 63/100
100/100 [==============================] - 612s 6s/step - loss: 0.3033 - accuracy: 0.8903 - val_loss: 0.8775 - val_accuracy: 0.733